This preprocessing notebook takes 6-hourly AR data from tARget v3, and selects only the times where an AR crosses the 1 km threshold in HMA.

Using the trackID from 

### Import

In [1]:
# Standard Python modules
import os, sys
import numpy as np
import pandas as pd
import xarray as xr
from datetime import datetime, timedelta
import metpy.calc as mpcalc
from metpy.units import units

# import personal modules

# Path to modules
sys.path.append('../modules')

# Import my modules
from ar_funcs import get_topo_mask
from timeseries import select_months_ds, select_months_df

In [2]:
# Set up paths
server = 'great'
path_to_data = '/home/nash/DATA/data/'                                      # project data -- read only
path_to_out  = '../out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '../figs/'      # figures

### Get list of AR dates and trackIDs when an AR crosses 1000 m elevation threshold in HMA

In [3]:
# identify ARs using single bound box with elevation mask during DJF
bbox = [20, 40, 65, 97] # HMA region
elev_thres = 1000.
ssn = 'DJF'

if ssn == 'DJF':
    start_date = '1979-12-01 0:00'
    end_date = '2015-02-28 18:00'
    start_mon = 12
    end_mon = 2
if ssn == 'MAM':
    start_date = '1980-03-01 0:00'
    end_date = '2014-05-31 18:00'
    start_mon = 3
    end_mon = 5

# open ds
filename =  path_to_data + 'ar_catalog/globalARcatalog_ERA-Interim_1979-2019_v3.0.nc'
ds = xr.open_dataset(filename, chunks={'time': 1460}, engine='netcdf4')
ds = ds.squeeze()
# remove lev and ens coords
ds = ds.reset_coords(names=['lev', 'ens'], drop=True)

# select lats, lons, and dates within start_date, end_date and months
lat1, lat2, lon1, lon2 = bbox
ds = ds.sel(time=slice(start_date, end_date), lat=slice(lat1,lat2), lon=slice(lon1,lon2))
ds = select_months_ds(ds, start_mon, end_mon)

# add topo mask
mask = get_topo_mask(ds.lat, ds.lon) # create a elevation dataset with same grid spacing as ds
ds = ds.where(mask.bedrock >= elev_thres) # mask ds where elevation is less than 1000 m

# convert dataset to dataframe
df = ds.kidmap.to_dataframe(dim_order=['time', 'lat', 'lon'])
df = df.dropna(axis='rows')
# keep only rows that have trackID
trackID = df.groupby('time').kidmap.unique()
# trackID # this is all trackIDs that crossed the 1000 m threshold


In [4]:
id_df = trackID.to_frame() # converts to a pandas dataframe
id_df = id_df.reset_index() # reset the index
id_df = id_df.rename(columns={'time': 'date'}) # rename time column into date
id_df = id_df.set_index(pd.to_datetime(id_df['date'])) # reset the index as "date"
id_df.index = id_df.index.strftime("%Y-%m-%d") # make it so the index date is normalized to daily
id_df = id_df.rename(columns={'date': 'time'}) # rename the date column back to time
id_df = id_df.reset_index() # remove the index
id_df = id_df.explode('kidmap') # explode the dataframe based on trackID
# id_df

In [5]:
# load AR CAT (from Nash et al. 2021)
filepath = path_to_out + 'AR-types_ALLDAYS.csv'
ar_cat = pd.read_csv(filepath)
ar_cat = ar_cat.rename(columns={'Unnamed: 0': 'date'})
ar_cat = ar_cat.set_index(pd.to_datetime(ar_cat['date']))
ar_cat = select_months_df(ar_cat, start_mon, end_mon)
ar_cat.index = ar_cat.index.strftime("%Y-%m-%d")
ar_cat = ar_cat.drop(columns=['date'])
ar_cat = ar_cat.reset_index()
idx = ar_cat['AR_CAT'] > 0
ar_cat = ar_cat.loc[idx]

# ar_cat

In [6]:
# merge id_df with ar_cat
merge_ar = pd.merge(id_df, ar_cat, how='outer', on='date')
track_ids = merge_ar.kidmap.unique() # get unique list of AR track IDs
ar_dates = merge_ar.time.unique() # get unique list of AR date/times (for later)
# merge_ar

In [7]:
# create df with trackID, ar_cat, start date, end date, and duration of AR (how long it is within HMA region)
ar = []
data = []
for i in [1, 2, 3]:
    idx = (merge_ar.AR_CAT == i)
    ar = merge_ar.loc[idx]

    for j, ids in enumerate(track_ids):
        idx = (ar.kidmap == ids)
        tmp = ar.loc[idx]
        start = pd.to_datetime(tmp.time.min())
        stop = pd.to_datetime(tmp.time.max()) + timedelta(hours=6)
        tmp = (stop - start)
        duration = tmp.total_seconds()/(3600) # convert to number of hours

        data.append([ids, i, start, stop, duration])
    
duration_df = pd.DataFrame(data, columns=['trackID', 'ar_cat', 'start_date', 'end_date', 'duration'])
duration_df = duration_df.dropna()
duration_df

,trackID,ar_cat,start_date,end_date,duration
1,2861.0,1,1979-12-02 00:00:00,1979-12-02 18:00:00,18.0
2,2871.0,1,1979-12-09 06:00:00,1979-12-09 18:00:00,12.0
3,2975.0,1,1979-12-16 12:00:00,1979-12-17 00:00:00,12.0
4,2988.0,1,1979-12-21 00:00:00,1979-12-22 12:00:00,36.0
5,3026.0,1,1979-12-24 06:00:00,1979-12-24 12:00:00,6.0
...,...,...,...,...,...
2367,114217.0,3,2015-01-03 18:00:00,2015-01-04 00:00:00,6.0
2373,114363.0,3,2015-01-21 06:00:00,2015-01-22 06:00:00,24.0
2374,114402.0,3,2015-01-29 12:00:00,2015-01-30 18:00:00,30.0
2379,114602.0,3,2015-02-24 00:00:00,2015-02-27 06:00:00,78.0


### Landslide DF

In [8]:
def expand_grid(lat,lon):
    '''list all combinations of lats and lons using expand_grid(lat,lon)'''
    test = [(A,B) for A in lat for B in lon]
    test = np.array(test)
    test_lat = test[:,0]
    test_lon = test[:,1]
    full_grid = pd.DataFrame({'lat': test_lat, 'lon': test_lon})
    full_grid = full_grid.sort_values(by=['lat','lon'])
    full_grid = full_grid.reset_index(drop=True)
    return full_grid

In [9]:
fname = path_to_data + 'CH2_generated_data/Global_Landslide_Catalog_Export.csv' #TODO check this - is it the raw downloaded data?
landslide = pd.read_csv(fname)

# Select lat/lon grid
lonmin = 65
lonmax = 100
latmin = 20
latmax = 42

## Select Landslides within Southern Asia region
idx = (landslide.latitude >= latmin) & (landslide.latitude <= latmax) & (landslide.longitude >= lonmin) & (landslide.longitude <= lonmax)
landslide = landslide.loc[idx]
# set event time as index
landslide = landslide.set_index(pd.to_datetime(landslide.event_date))
# landslide.index = landslide.index.normalize()

# select only landslide dates that are between december and may
idx = (landslide.index.month >= 12) | (landslide.index.month <= 5)
landslide = landslide[idx]

# rename and reindex
landslide = landslide.rename(columns={"latitude": "lat", "longitude": "lon", "event_date": "event_time"})
landslide = landslide.reset_index()

# round event time to the nearest 6 hours
landslide['time'] = landslide['event_date'].dt.round('6H')
landslide = landslide.set_index(pd.to_datetime(landslide.time))

# select only landslide dates that are between december and may
idx = (landslide.index.month >= 12) | (landslide.index.month <= 5)
landslide = landslide[idx]

# landslide

In [10]:
# now we want to see if there is an AR present at the same time and location as the landslides
# open the trackID for ARs
filename =  path_to_data + 'ar_catalog/globalARcatalog_ERA-Interim_1979-2019_v3.0.nc'
ar = xr.open_dataset(filename, engine='netcdf4')
ar = ar.squeeze()

# Select months
idx = (ar.time.dt.month >= 12) | (ar.time.dt.month <= 5)
kid = ar.kidmap.sel(time=idx) # trackID for indexing

# slice the dates so both ds match
kid = kid.sel(time=slice('1979-12-01 00', '2019-05-31 00:00'))
# kid

In [11]:
## for each landslide_id, if the lat/lon falls within an AR, keep that AR ID and landslide ID
landslideID = []
arID = []
landslide_lat = []
landslide_lon = []
for i, row in landslide.T.iteritems():
    t = kid.sel(lat=row['lat'], lon=row['lon'], time=row['time'], method='nearest').values
    # print(t)
    if t > 0:
        landslideID.append(row['event_id'])
        arID.append(t)
        landslide_lat.append(row['lat'])
        landslide_lon.append(row['lon'])
        
d = {'landslideID': landslideID, 'trackID': arID, 
     'landslide_lat': landslide_lat, 'landslide_lon': landslide_lon}
landslide_df = pd.DataFrame(data=d)
# convert the dtype for the trackID column
landslide_df = landslide_df.astype({'trackID': 'float64'})

# landslide_df

In [12]:
# merge AR duration df and landslide DF
merged_data = pd.merge(duration_df, landslide_df, how='outer', on='trackID')
# merged_data 
# note the rows that do not have a date or time 
# are landslides that are associated with a specific AR that was not considered a "HMA AR"

In [13]:
## test to make sure merged correctly
# idx = merged_data.landslideID > 0
# test = merged_data[idx]
# test

In [14]:
# drop the rows that are not a HMA AR
idx = merged_data['ar_cat'] > 0
merged_data = merged_data.loc[idx]
merged_data

,trackID,ar_cat,start_date,end_date,duration,landslideID,landslide_lat,landslide_lon
0,2861.0,1.0,1979-12-02 00:00:00,1979-12-02 18:00:00,18.0,NaN,NaN,NaN
1,2861.0,2.0,1979-12-01 00:00:00,1979-12-02 00:00:00,24.0,NaN,NaN,NaN
2,2871.0,1.0,1979-12-09 06:00:00,1979-12-09 18:00:00,12.0,NaN,NaN,NaN
3,2871.0,2.0,1979-12-08 06:00:00,1979-12-09 00:00:00,18.0,NaN,NaN,NaN
4,2975.0,1.0,1979-12-16 12:00:00,1979-12-17 00:00:00,12.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
983,114196.0,3.0,2014-12-31 18:00:00,2015-01-03 12:00:00,66.0,NaN,NaN,NaN
984,114217.0,3.0,2015-01-03 18:00:00,2015-01-04 00:00:00,6.0,NaN,NaN,NaN
985,114363.0,3.0,2015-01-21 06:00:00,2015-01-22 06:00:00,24.0,NaN,NaN,NaN
986,114402.0,3.0,2015-01-29 12:00:00,2015-01-30 18:00:00,30.0,NaN,NaN,NaN


In [52]:
def ar_ivt(df, ds, domains, clim_mean, clim_std):
    '''Calculate maximum IVT for a subregion in a ds and append to dataframe.
     For each range of AR event dates, we find the maximum IVT for the duration of the AR for every grid cell. 
    '''
    # the final IVT statistic to retain
    ivtdir_vals = []
    ivt_vals = []
    freeze_vals = []
    # loop through each AR track
    for i, (arcat, track) in enumerate(zip(df.ar_cat.values, df.trackID.values)):
        start = df.start_date.values[i]
        end = df.end_date.values[i]
        # print('Getting maximum between', start, end)
        print(start, end)
        # get bbox based on ar_cat
        bnds = domains[int(arcat)-1]
        # select only the time steps for AR event and specified domain
        tmp = ds.sel(time=slice(start, end), lat=slice(bnds[2], bnds[3]), lon=slice(bnds[0], bnds[1]))
        
        ## remove climatology from freezing level 
        clim_ar = clim_mean.sel(AR_CAT = arcat, lat=slice(bnds[2], bnds[3]), lon=slice(bnds[0], bnds[1]))
        std_ar = clim_std.sel(AR_CAT = arcat, lat=slice(bnds[2], bnds[3]), lon=slice(bnds[0], bnds[1]))
        tmp['z_new'] = (tmp.z - clim_ar.z)/std_ar.z # standardized anomalies
        ## average freezing level over time, lat, lon
        freeze = tmp['z_new'].max(['time'], skipna=True).mean(['lat', 'lon']).values
        freeze_vals.append(freeze.tolist())
        
        ### localized IVT maxima during event
        # event_max = tmp.where(tmp.ivt==tmp.ivt.max(), drop=True).squeeze()
        event_max = tmp.where(tmp.ivt==tmp.ivt.max(), drop=True).squeeze().load() # this was taking too long, decided to load earlier
        ## pull IVT and IVTDIR where ivt is max
        uvec = event_max.ivtu.values
        uvec = units.Quantity(uvec, "m/s")
        vvec = event_max.ivtv.values
        vvec = units.Quantity(vvec, "m/s")
        ivtdir = mpcalc.wind_direction(uvec, vvec)
        ivtdir_vals.append(ivtdir.item())
        ivt_vals.append(event_max.ivt.values.tolist())
        
        # # pull freezing level anomaly where ivt is max
        # freeze_vals.append(event_max.z_new.values)
        
    final = [ivtdir_vals, ivt_vals, freeze_vals]
        
    return final

def ar_precip(df, ds, domains, mode):
    '''Calculate precipitation statistics for a subregion in a ds and append to dataframe.
     Mode is chosen based on calculation. For each range of AR event dates, we calculate the total accumulated precip for every grid cell. 
     Then we remove all gridcells that had less than 1 mm of rain per event (these are not included in any calc)
     Then we weight the gridcells by the cosine of the latitude.
     Then based on mode selected, different statistics are retained:
         'mean-total' averages all viable gridcells within the subregion and retains this number
         'max-total' selects the maximum gridcell value to append
         'percentile-total' calcuates the 95th percentile and then averages all the grid cells that exceed this threshold
    '''
    # the final precip statistic to retain
    m1_vals = []

    for i, (arcat, track) in enumerate(zip(df.ar_cat.values, df.trackID.values)):
        start = df.start_date.values[i]
        end = df.end_date.values[i]
        # print('Getting maximum between', start, end)
        print(i)
        # get bbox based on ar_cat
        bnds = domains[int(arcat)-1]
        # select only the time steps for AR event and specified domain
        tmp = ds.sel(time=slice(start, end), lat=slice(bnds[2], bnds[3]), lon=slice(bnds[0], bnds[1]))

        ### event-total precipitation per event for every grid cell
        tmp = tmp.sum('time')
        ### mask out grid cells with less than 1 mm per event
        tmp2 = xr.where(cond=(tmp.prec > 1), x=tmp.prec, y=np.nan)

        ### area weighted
        # tmp = tmp2.weighted(tmp.weights)

        if mode == 'mean-total':
            ## mode 1: mean-total
            # average over gridcells in weighted subregion
            mean_tot = tmp.mean(['lat', 'lon'], skipna=True)
            # append to list
            m1_vals.append(mean_tot.values.tolist())
        elif mode == 'max-total':
            ## mode 2: max-total
            ### localized precip maxima during event
            event_max = tmp2.max(['lat', 'lon'])
            m1_vals.append(event_max.values.tolist())
        elif mode == 'percentile-total':
            ## mode 3: percentile-total
            ###  get 95th percentile thres
            q_thres = tmp2.quantile(0.95, dim=['lat', 'lon'], interpolation='linear')
            ## mask out grid cells below threshold
            perc_prec = xr.where(cond=(tmp2 > q_thres), x=tmp2, y=np.nan)
            # average over all grid cells skipping nans
            mean = perc_prec.mean(['lat', 'lon'], skipna=True)
            m1_vals.append(mean.values.tolist())

        
    return m1_vals

## load 2D WRF data

### get lats and lons from d01 and d02

In [16]:
## pull wrflats and wrflons from first file
fname = path_to_data + 'wrf_hasia/d01/ivt/3hr/tmp_2015.nc'
tmp = xr.open_dataset(fname)
# print(tmp.time[:100])
# print(tmp.time[-100:])

## assign those lats to the other ds when you loop
wrflats = tmp.lat.values
wrflons = tmp.lon.values

fname = path_to_data + 'wrf_hasia/d02/prec/3hr/tmp_2014.nc'
tmp = xr.open_dataset(fname)
# print(tmp.time[:100])
# print(tmp.time[-100:])

## assign those lats to the other ds when you loop
wrflats2 = tmp.lat.values
wrflons2 = tmp.lon.values


In [17]:
%%time
varname = 'zerodegisotherm'
domain = 'd01'

filename_pattern = path_to_data + 'wrf_hasia/{0}/{1}/daily/out.wrf6km.{1}.daily_*.nc'.format(domain, varname)
print(filename_pattern)
ds = xr.open_mfdataset(filename_pattern)

# Trim date range
idx = slice(start_date, end_date)
ds = ds.sel(time=idx)

# select only months we are interested in
ds = select_months_ds(ds, start_mon, end_mon)

ds

/home/nash/DATA/data/wrf_hasia/d01/zerodegisotherm/daily/out.wrf6km.zerodegisotherm.daily_*.nc
CPU times: user 580 ms, sys: 111 ms, total: 691 ms
Wall time: 3.24 s


<xarray.Dataset>
Dimensions:  (time: 3249, lat: 249, lon: 259)
Coordinates:
  * time     (time) datetime64[ns] 1979-12-01 1979-12-02 ... 2015-02-28
  * lat      (lat) float32 16.43 16.63 16.83 17.02 ... 55.8 55.92 56.04 56.15
  * lon      (lon) float32 60.2 60.41 60.62 60.83 ... 113.2 113.4 113.6 113.8
Data variables:
    z        (time, lat, lon) float32 dask.array<chunksize=(31, 249, 259), meta=np.ndarray>

In [24]:
%%time
def preprocess_ivt(ds):
    '''keep only the current year'''
    year = ds.time.dt.year.max().values
    ds = ds.assign_coords({"lon": wrflons, "lat": wrflats})
    if year == 1980:
        ds = ds
    else:
        ds = ds.sel(time=slice('{0}-01-01 00:00'.format(year), '{0}-12-31 21:00'.format(year)))
    return ds

def preprocess_prec(ds):
    '''keep only the current year'''
    year = ds.time.dt.year.max().values
    ds = ds.assign_coords({"lon": wrflons2, "lat": wrflats2})
    if year == 1980:
        ds = ds
    else:
        ds = ds.sel(time=slice('{0}-01-01 00:00'.format(year), '{0}-12-31 21:00'.format(year)))
    return ds

domains = ['d01', 'd02', 'd01']
varname_lst = ['ivt', 'prec', 'zerodegisotherm']

## loop through each ds
ds_lst = []
for i, (dom, varname) in enumerate(zip(domains, varname_lst)):
    print('Reading...', varname)
    if server == 'great':
        data_path = path_to_data + 'wrf_hasia/'
    else:
        data_path = path_to_data + 'wrf_preprocessed_data/wrf_6km/'
        
    filename_pattern = '{0}/{1}/3hr/tmp_*.nc'.format(dom, varname)
    fname = data_path + filename_pattern
    
    if varname == 'ivt':
        ds = xr.open_mfdataset(fname, preprocess=preprocess_ivt)
        ds = ds.assign(ivt=lambda ds: np.sqrt(ds.ivtu**2 + ds.ivtv**2))
    elif varname == 'prec':
        ds = xr.open_mfdataset(fname, preprocess=preprocess_prec)
        ## shift subtraction to get mm per hour 
        # # rain at next time step - rain at current time step
        ds = ds.shift(time=-1) - ds # if in xarray
    elif varname == 'geopotential':
        ds = ds.sel(lev=250.)
    elif varname == 'zerodegisotherm':
        ds = xr.open_mfdataset(fname, preprocess=preprocess_ivt)
    
    # subset to just ar days
    # ds = ds.sel(time = slice(start_date, end_date))
    # ds = select_months_ds(ds, start_mon, end_mon)
    # ds = ds.sel(time = ar_dates[:-1])
    
    ds_lst.append(ds)
    
ivt = ds_lst[0]
prec = ds_lst[1]
zerodeg = ds_lst[2]

wrf_d01 = xr.combine_by_coords([ds_lst[0], ds_lst[2]]) ## combine IVT and freezing level into 1 ds

Reading... ivt
Reading... prec
Reading... zerodegisotherm
CPU times: user 2.3 s, sys: 102 ms, total: 2.4 s
Wall time: 2.4 s


In [25]:
## Having trouble with wrf ds not having all the dates in the ar_dates list 
## use this to find out which dates are having a problem
# make a pandas dataframe of AR Dates
d = {'dates': ar_dates[:-1]}
df_A = pd.DataFrame(data=d)
df_A = df_A.set_index(pd.to_datetime(df_A['dates'])) # reset the index as "dates"


# make a pandas dataframe of WRF dates
d = {'dates': zerodeg.time}
df_B = pd.DataFrame(data=d)
df_B = df_B.set_index(pd.to_datetime(df_B['dates'])) # reset the index as "dates"

# test = df_A.isin(df_B)

x = df_A.index
y = df_B.index
test = x.isin(y)

idx = (test== False)
df_A.loc[idx]


,dates
dates,


In [53]:
%%time

# latmin, latmax, lonmin, lonmax
ext1 = [71, 79, 32, 37] # Western precip anomalies
ext2 = [69, 74, 37, 40] # Northwestern precip anomalies
ext3 = [90, 99, 24, 30] # Eastern precip anomalies

region_name = ['western', 'northwestern', 'eastern']
domains = [ext1, ext2, ext3]
clim_mean = xr.open_dataset('/home/sbarc/students/nash/data/wrf_hasia/d01/zerodegisotherm_ar_clim_new.nc') # freezing level climatology
clim_std = xr.open_dataset('/home/sbarc/students/nash/data/wrf_hasia/d01/zerodegisotherm_ar_std_new.nc') # freezing level standard deviation

print('Processing IVT and freezing level...')
## For each row, calculate the maximum IVT within the region between start and end
ivt_final = ar_ivt(merged_data.iloc[0:3], wrf_d01, domains, clim_mean, clim_std)

## For each row, calculate the maximum precip within the region between start and end
prec_final = ar_precip(merged_data.iloc[0:3], prec, domains, 'max-total')

## attach data to existing df
# merged_data['ivt'] = ivt_final[1]
# merged_data['ivtdir'] = ivt_final[0]
# merged_data['freeze'] = ivt_final[2]
# merged_data['prec'] = prec_final

Processing IVT and freezing level...
1979-12-02T00:00:00.000000000 1979-12-02T18:00:00.000000000


/home/sbarc/students/nash/miniconda3/envs/hasia2/lib/python3.7/site-packages/dask/utils.py:35: RuntimeWarning: All-NaN slice encountered
  return func(*args, **kwargs)
/home/sbarc/students/nash/miniconda3/envs/hasia2/lib/python3.7/site-packages/dask/core.py:121: RuntimeWarning: All-NaN slice encountered
  return func(*(_execute_task(a, cache) for a in args))


1979-12-01T00:00:00.000000000 1979-12-02T00:00:00.000000000


/home/sbarc/students/nash/miniconda3/envs/hasia2/lib/python3.7/site-packages/dask/utils.py:35: RuntimeWarning: All-NaN slice encountered
  return func(*args, **kwargs)
/home/sbarc/students/nash/miniconda3/envs/hasia2/lib/python3.7/site-packages/dask/core.py:121: RuntimeWarning: All-NaN slice encountered
  return func(*(_execute_task(a, cache) for a in args))


1979-12-09T06:00:00.000000000 1979-12-09T18:00:00.000000000


/home/sbarc/students/nash/miniconda3/envs/hasia2/lib/python3.7/site-packages/dask/utils.py:35: RuntimeWarning: All-NaN slice encountered
  return func(*args, **kwargs)
/home/sbarc/students/nash/miniconda3/envs/hasia2/lib/python3.7/site-packages/dask/core.py:121: RuntimeWarning: All-NaN slice encountered
  return func(*(_execute_task(a, cache) for a in args))


0
1
2
CPU times: user 3.96 s, sys: 5.47 s, total: 9.43 s
Wall time: 9.39 s


In [54]:
ivt_final[2]

[1.5622614622116089, 1.7191874980926514, 1.795113205909729]

In [ ]:
## load filtered annual climatology and std
clim_std = xr.open_dataset(path_to_data + 'wrf_hasia/d01/zerodegisotherm/daily_std_clim_zerodegisotherm.nc')
clim_mean = xr.open_dataset(path_to_data + 'wrf_hasia/d01/zerodegisotherm/filtered_daily_mean_clim_zerodegisotherm.nc')


clim_mean = clim_mean.sel(AR_CAT = 1)

## Calculate Anomalies
anomalies = ds.groupby('time.dayofyear') - clim_mean

# normalize AR dates
## get normalized start date for each row in the df - need this to create subset list of freezing level days
df = merged_data.rename(columns={'start_date': 'date'})
df = df.set_index(pd.to_datetime(df['date']))
df = select_months_df(df, start_mon, end_mon)
df.index = df.index.strftime("%Y-%m-%d")
df = df.rename(columns={'date': 'start_date'})
df = df.reset_index()
df

# get list of dates that ar is present
ar_dates = pd.to_datetime(df['date']).values
# subset freezing level to just ar days
anomalies = anomalies.sel(time = ar_dates)
anomalies

## Calculate low freezing (x - mean < - 1.5*std)
low_freezing = anomalies.where(anomalies.z.groupby('time.dayofyear') < clim_std.z*-1.)
## Calculate high freezing (x - mean > 1.5*std)
high_freezing = anomalies.where(anomalies.z.groupby('time.dayofyear') > clim_std.z*1.)

# make a ds for each subregion
ds_low = []
ds_high = []
for i, dom in enumerate(domains):
    tmp = low_freezing.sel(lon=slice(dom[0], dom[1]), lat=slice(dom[2], dom[3]))
    ds_low.append(tmp.load())
    tmp = high_freezing.sel(lon=slice(dom[0], dom[1]), lat=slice(dom[2], dom[3]))
    ds_high.append(tmp.load())
ds_low

In [25]:
# %%time
# # make a ds for each subregion
# ds_lst = []
# for i, bnds in enumerate(domains):
#     tmp = ivt.sel(lat=slice(bnds[2], bnds[3]), lon=slice(bnds[0], bnds[1])) 
#     ds_lst.append(tmp)
# ds_lst

In [ ]:
%%time
## this version takes the average value in the subregion
for i, region in enumerate(region_name):
    ## compute low freezing level
    x = ds_low[i].z.values
    # flatten array to 2D so it is ntimes, nlat*nlon
    ntimes, nlats, nlons = x.shape
    x = x.reshape(ntimes, nlats*nlons)
    
    # calculate mean, skipping nans
    low = np.nanmean(x, axis=1)
    colname = region + '_low'
    df[colname] = low

    ## compute high freezing level
    x = ds_high[i].z.values
    # flatten array to 2D so it is ntimes, nlat*nlon
    ntimes, nlats, nlons = x.shape
    x = x.reshape(ntimes, nlats*nlons)

    # count number of True for each time step
    high = np.nanmean(x, axis=1)
    colname = region + '_high'
    df[colname] = high


df

## this version counts the number of grid cells in ds_high and ds_low
## if the number of ds_high gricells > ds_low gridcells = above average day
## if the number of ds_low > ds_high gricells = below average day
for i, region in enumerate(region_name):
    ## compute low freezing level
    x = ds_low[i].z.values
    # flatten array to 2D so it is ntimes, nlat*nlon
    ntimes, nlats, nlons = x.shape
    x = x.reshape(ntimes, nlats*nlons)

    # mark True if value is not nan
    a = ~np.isnan(x)
    # # mark True if any value for each time step is True (aka not nan)
    # z = np.any(a, axis=1)
    
    # count number of True for each time step
    low = np.count_nonzero(a, axis=1)
    df['low'] = low

    ## compute high freezing level
    x = ds_high[i].z.values
    # flatten array to 2D so it is ntimes, nlat*nlon
    ntimes, nlats, nlons = x.shape
    x = x.reshape(ntimes, nlats*nlons)

    # mark True if value is not nan
    a = ~np.isnan(x)
    # # mark True if any value for each time step is True (aka not nan)
    # z = np.any(a, axis=1)
    
    # count number of True for each time step
    high = np.count_nonzero(a, axis=1)
    df['high'] = high
    
    colname = region + '_freeze'
    df[colname] = 0
    df.loc[df['low'] > df['high'], colname] = -1
    df.loc[df['low'] < df['high'], colname] = 1
    
    # drop low and high columns
    df = df.drop(columns=['low', 'high'])


df

In [ ]:
# Export dataframes as csv
df.to_csv(path_to_out + '{0}_ivt_ar_types_freezing_level_max_prec_new.csv'.format(ssn))